# Keyswitch Analytics

By Kenneth Burchfiel

Released under the MIT license

This notebook demonstrates how to compare results for different keyswitch options. This notebook can easily be modified to compare other elements of your setting (such as your location) instead.

(More documentation to come!)

In [1]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
pd.set_option('display.max_columns', 1000)

In [2]:
df_results = pd.read_csv('results.csv')

df_results

,Test_Number,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms),Backspace % Last 10 Avg,Backspace % Last 100 Avg,Backspace % Last 1000 Avg,Session,Test_#_Within_Session,Incorrect Character % Last 10 Avg,Incorrect Character % Last 100 Avg,Incorrect Character % Last 1000 Avg,Test Version,Location,Keyboard,Layout,Caffeine,Custom_1,Custom_2,Custom_3,Autostart,% of Words Typed Correctly,Extra Keystrokes %,Extra Keystrokes % Last 10 Avg,Extra Keystrokes % Last 100 Avg,Extra Keystrokes % Last 1000 Avg
0,1,1.698121e+09,2023-10-24 00:17:38.017017,2023-10-24 04:17:38.017017+00:00,56,6.571754,8.521317,102.255799,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,102.256,2023,10,0,1,2023-10-24,17,2,2,1,2023-10-24 00:17:44.588771456,2023,10,2023-10-24,0,17,2,2,1,NaN,8769,7.363973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.698121e+09,2023-10-24 00:18:06.165502,2023-10-24 04:18:06.165502+00:00,135,13.997536,9.644554,115.734653,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,108.995,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:20.163038180,2023,10,2023-10-24,0,18,2,2,1,NaN,7540,20.348653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1.698121e+09,2023-10-24 00:18:24.381290,2023-10-24 04:18:24.381290+00:00,56,4.714291,11.878774,142.545292,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,120.179,2023,10,0,1,2023-10-24,18,2,2,1,2023-10-24 00:18:29.095581096,2023,10,2023-10-24,0,18,2,2,1,NaN,3214,66.053883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1.698121e+09,2023-10-24 00:21:24.331389,2023-10-24 04:21:24.331389+00:00,56,4.878704,11.478458,137.741497,GEN,1,1,"In the beginning, God created the heavens and ...",NaN,NaN,NaN,NaN,124.569,2023,10,0,1,2023-10-24,21,3,2,1,2023-10-24 00:21:29.210093071,2023,10,2023-10-24,0,21,2,3,1,NaN,4033,57.400951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1.698121e+09,2023-10-24 00:22:03.749152,2023-10-24 04:22:03.749152+00:00,135,19.078445,7.076048,84.912580,GEN,1,2,The earth was formless and empty. Darkness was...,NaN,NaN,NaN,NaN,116.638,2023,10,0,1,2023-10-24,22,3,2,1,2023-10-24 00:22:22.827597200,2023,10,2023-10-24,0,22,2,3,1,NaN,9331,1.426307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9460,9461,1.710173e+09,2024-03-11 11:58:33.048464,2024-03-11 15:58:33.048464+00:00,77,8.535126,9.021543,108.258512,JHN,6,30605,"Jesus said therefore to the twelve, ""You don't...",67.0,133.762684,139.960245,129.587784,133.177,2024,3,11,1,2024-03-11,58,6,4,2,2024-03-11 11:58:41.583589600,2024,3,2024-03-11,11,58,4,6,2,0.0,8319,12.118331,2.597403,7.792208,NaN,NaN,NaN,NaN,NaN,NaN,203,126,6.526332,6.125440,7.263518,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,86.666667,10.389610,11.142081,10.531615,NaN
9461,9462,1.710173e+09,2024-03-11 11

The following cell stores a list of start and end times for each trial. 

In [3]:
trial_start_and_end_times_list = [('2024-03-11 11:25', '2024-03-11 11:33:04'),
                                  ('2024-03-11 11:33:06', '2024-03-11 11:56')]


The next cell uses a list comprehension to create a filtered copy of df_results that includes only results that appear *after* the first timestamp in the tuple and *before* the second timestamp. (In order words, results that match the timestamps in the list exactly won't get included.)

In [4]:
df_trial_results = pd.concat(
[df_results.query(f"Local_Start_Time > '{time_pair[0]}' \
& Local_Start_Time < '{time_pair[1]}'").copy() for time_pair 
in trial_start_and_end_times_list]) # This code, for each set of timestamps
# within trial_start_and_end_times_list, creates a copy of df_results
# that only includes results after the first timestamp and before the second 
# one. It next combines these copies together via pd.concat().

df_trial_results

,Test_Number,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms),Backspace % Last 10 Avg,Backspace % Last 100 Avg,Backspace % Last 1000 Avg,Session,Test_#_Within_Session,Incorrect Character % Last 10 Avg,Incorrect Character % Last 100 Avg,Incorrect Character % Last 1000 Avg,Test Version,Location,Keyboard,Layout,Caffeine,Custom_1,Custom_2,Custom_3,Autostart,% of Words Typed Correctly,Extra Keystrokes %,Extra Keystrokes % Last 10 Avg,Extra Keystrokes % Last 100 Avg,Extra Keystrokes % Last 1000 Avg
9335,9336,1.710171e+09,2024-03-11 11:26:37.472366,2024-03-11 15:26:37.472366+00:00,192,16.778450,11.443250,137.319004,JHN,5,30510,"Jesus therefore answered them, ""Most certainly...",19.0,137.585592,138.500531,128.903824,133.048,2024,3,11,1,2024-03-11,26,3,2,1,2024-03-11 11:26:54.250815700,2024,3,2024-03-11,11,26,2,3,1,0.0,4108,56.608558,3.125000,8.333333,NaN,NaN,NaN,NaN,NaN,NaN,203,1,6.676314,5.521322,7.532080,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,88.235294,13.541667,12.194104,NaN,NaN
9336,9337,1.710171e+09,2024-03-11 11:26:56.220335,2024-03-11 15:26:56.220335+00:00,152,11.056986,13.746965,164.963585,JHN,5,30511,"For the Father has affection for the Son, and ...",20.0,140.528065,138.713826,128.928716,133.052,2024,3,11,1,2024-03-11,26,3,2,1,2024-03-11 11:27:07.277320700,2024,3,2024-03-11,11,27,2,3,1,0.0,586,93.819334,1.315789,4.605263,NaN,NaN,NaN,NaN,NaN,NaN,203,2,6.901546,5.540529,7.531622,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,96.551724,5.921053,12.315621,NaN,NaN
9337,9338,1.710171e+09,2024-03-11 11:27:11.907570,2024-03-11 15:27:11.907570+00:00,106,7.391912,14.339997,172.079964,JHN,5,30512,For as the Father raises the dead and gives th...,21.0,142.400652,138.888181,128.988686,133.056,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:19.299482300,2024,3,2024-03-11,11,27,2,3,1,1.0,257,97.295298,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,203,3,6.300453,5.540529,7.515985,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,100.000000,0.000000,11.168080,NaN,NaN
9338,9339,1.710171e+09,2024-03-11 11:27:21.423686,2024-03-11 15:27:21.423686+00:00,71,6.279210,11.307155,135.685859,JHN,5,30513,"For the Father judges no one, but he has given...",22.0,143.358556,138.959112,128.984768,133.056,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:27.702896000,2024,3,2024-03-11,11,27,2,3,1,0.0,4387,53.660856,1.408451,1.408451,NaN,NaN,NaN,NaN,NaN,NaN,203,4,5.989321,5.501583,7.511974,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,93.333333,2.816901,10.376323,NaN,NaN
9339,9340,1.710171e+09,2024-03-11 11:27:31.307316,2024-03-11 15:27:31.307316+00:00,126,10.224435,12.323420,147.881036,JHN,5,30514,"that all may honor the Son, even as they honor...",23.0,147.057733,139.134097,128.998703,133.058,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:41.531750700,2024,3,2024-03-11,11,27,2,3,1,0.0,2351,75.171685,1.587302,4.761905,NaN,NaN,NaN,NaN,NaN,NaN,203,5,5.024833,5.513488,7.514355,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,96.000000,9.523810,8.701586,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9451,9452,1.710173e+09,2024-03-11 11:55:05.122081,2024-03-11 15:55:05.122081+00:00,165,12.769395,12

Here's an alternative version of the previous cell that uses a for loop rather than a list comprehension. It's a bit easier to interpret but requires more lines of code, so I prefer the previous version.

In [5]:
# trial_results_df_list = []
# for time_pair in trial_start_and_end_times_list:
#     df_selected_results = df_results.query(
#         f"Local_Start_Time > '{time_pair[0]}' \
# & Local_Start_Time < '{time_pair[1]}'").copy()
#     trial_results_df_list.append(df_selected_results)

# df_trial_results = pd.concat([df for df in trial_results_df_list])
# df_trial_results

In [6]:
df_keyswitch_accuracy_and_wpm_comparison = px.scatter(df_trial_results, x= 'Extra Keystrokes %', 
y = 'WPM', trendline = 'ols', color = 'Keyboard', color_discrete_map = {
    'cherry red':'red', 'cherry mx2a brown':'brown', 'cherry mx2a black':'black'})
df_keyswitch_accuracy_and_wpm_comparison.write_html('keyswitch_accuracy_and_wpm_comparison.html')
df_keyswitch_accuracy_and_wpm_comparison

In [7]:
df_trial_results.pivot_table(
index = 'Keyboard', values = ['WPM', 'Extra Keystrokes %'], aggfunc = 'mean').reset_index(
).sort_values('WPM', ascending = False)

,Keyboard,Extra Keystrokes %,WPM
1,cherry mx2a brown,10.527581,144.719176
0,cherry mx2a black,8.001496,144.464398
2,cherry red,9.929853,140.558658


In [8]:
df_trial_results.query("`Extra Keystrokes %` == 0").pivot_table(
index = 'Keyboard', values = ['WPM'], aggfunc = 'mean').reset_index(
).sort_values('WPM', ascending = False)

,Keyboard,WPM
1,cherry mx2a brown,167.543974
0,cherry mx2a black,166.595277
2,cherry red,165.214394


Creating categorical variables for different keyboard layouts:

(These variables will be useful for upcoming regression analyses. Also Note that the original 'Keyboard' column is removed in the process of generating these categorical variables.)

In [9]:
df_trial_results_for_regression = pd.get_dummies(
data = df_trial_results, columns = ['Keyboard'], drop_first=True, dtype = 'int').copy() 
# For more on get_dummies(), see: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
df_trial_results

,Test_Number,Unix_Start_Time,Local_Start_Time,UTC_Start_Time,Characters,Seconds,CPS,WPM,Book,Chapter,Verse_Order,Verse,Verse #,Last 10 Avg,Last 100 Avg,Last 1000 Avg,cumulative_avg,Local_Start_Year,Local_Start_Month,Local_Start_Hour,Count,Local_Start_Date,Local_Start_Minute,Local_Start_10_Minute_Block,Local_Start_15_Minute_Block,Local_Start_30_Minute_Block,Local_End_Time,Local_End_Year,Local_End_Month,Local_End_Date,Local_End_Hour,Local_End_Minute,Local_End_15_Minute_Block,Local_End_10_Minute_Block,Local_End_30_Minute_Block,Mistake_Free_Test,WPM_Rank,WPM_Percentile,Backspaces as % of Verse Length,Incorrect Characters as % of Verse Length,Min Character Time (ms),Median Character Time (ms),Max Character Time (ms),Backspace % Last 10 Avg,Backspace % Last 100 Avg,Backspace % Last 1000 Avg,Session,Test_#_Within_Session,Incorrect Character % Last 10 Avg,Incorrect Character % Last 100 Avg,Incorrect Character % Last 1000 Avg,Test Version,Location,Keyboard,Layout,Caffeine,Custom_1,Custom_2,Custom_3,Autostart,% of Words Typed Correctly,Extra Keystrokes %,Extra Keystrokes % Last 10 Avg,Extra Keystrokes % Last 100 Avg,Extra Keystrokes % Last 1000 Avg
9335,9336,1.710171e+09,2024-03-11 11:26:37.472366,2024-03-11 15:26:37.472366+00:00,192,16.778450,11.443250,137.319004,JHN,5,30510,"Jesus therefore answered them, ""Most certainly...",19.0,137.585592,138.500531,128.903824,133.048,2024,3,11,1,2024-03-11,26,3,2,1,2024-03-11 11:26:54.250815700,2024,3,2024-03-11,11,26,2,3,1,0.0,4108,56.608558,3.125000,8.333333,NaN,NaN,NaN,NaN,NaN,NaN,203,1,6.676314,5.521322,7.532080,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,88.235294,13.541667,12.194104,NaN,NaN
9336,9337,1.710171e+09,2024-03-11 11:26:56.220335,2024-03-11 15:26:56.220335+00:00,152,11.056986,13.746965,164.963585,JHN,5,30511,"For the Father has affection for the Son, and ...",20.0,140.528065,138.713826,128.928716,133.052,2024,3,11,1,2024-03-11,26,3,2,1,2024-03-11 11:27:07.277320700,2024,3,2024-03-11,11,27,2,3,1,0.0,586,93.819334,1.315789,4.605263,NaN,NaN,NaN,NaN,NaN,NaN,203,2,6.901546,5.540529,7.531622,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,96.551724,5.921053,12.315621,NaN,NaN
9337,9338,1.710171e+09,2024-03-11 11:27:11.907570,2024-03-11 15:27:11.907570+00:00,106,7.391912,14.339997,172.079964,JHN,5,30512,For as the Father raises the dead and gives th...,21.0,142.400652,138.888181,128.988686,133.056,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:19.299482300,2024,3,2024-03-11,11,27,2,3,1,1.0,257,97.295298,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,203,3,6.300453,5.540529,7.515985,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,100.000000,0.000000,11.168080,NaN,NaN
9338,9339,1.710171e+09,2024-03-11 11:27:21.423686,2024-03-11 15:27:21.423686+00:00,71,6.279210,11.307155,135.685859,JHN,5,30513,"For the Father judges no one, but he has given...",22.0,143.358556,138.959112,128.984768,133.056,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:27.702896000,2024,3,2024-03-11,11,27,2,3,1,0.0,4387,53.660856,1.408451,1.408451,NaN,NaN,NaN,NaN,NaN,NaN,203,4,5.989321,5.501583,7.511974,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,93.333333,2.816901,10.376323,NaN,NaN
9339,9340,1.710171e+09,2024-03-11 11:27:31.307316,2024-03-11 15:27:31.307316+00:00,126,10.224435,12.323420,147.881036,JHN,5,30514,"that all may honor the Son, even as they honor...",23.0,147.057733,139.134097,128.998703,133.058,2024,3,11,1,2024-03-11,27,3,2,1,2024-03-11 11:27:41.531750700,2024,3,2024-03-11,11,27,2,3,1,0.0,2351,75.171685,1.587302,4.761905,NaN,NaN,NaN,NaN,NaN,NaN,203,5,5.024833,5.513488,7.514355,v2,home,cherry red,dvorak,yes,NaN,NaN,NaN,0.0,96.000000,9.523810,8.701586,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9451,9452,1.710173e+09,2024-03-11 11:55:05.122081,2024-03-11 15:55:05.122081+00:00,165,12.769395,12

In [10]:
keyboard_categorical_columns = [
column for column in df_trial_results_for_regression if 'Keyboard_' in column]
keyboard_categorical_columns

['Keyboard_cherry mx2a brown', 'Keyboard_cherry red']

In [11]:
y = df_trial_results_for_regression['WPM']
x = df_trial_results_for_regression['Extra Keystrokes %']
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WPM   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.698
Method:                 Least Squares   F-statistic:                     278.0
Date:                Mon, 11 Mar 2024   Prob (F-statistic):           6.44e-33
Time:                        13:36:26   Log-Likelihood:                -463.12
No. Observations:                 121   AIC:                             930.2
Df Residuals:                     119   BIC:                             935.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                159.0359      1.391    114.357      0.000     156.282     161.790
Extra Keystrokes %    -1.6616      0.100    -16.672      0.000      -1.859      -1.464
==============================================================================
Omnibus:                        0.227   Durbin-Watson:                   1.649
Prob(Omnibus):                  0.893   Jarque-Bera (JB):                0.037
Skew:                           0.024   Prob(JB):                        0.982
Kurtosis:                       3.071   Cond. No.                         19.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
y = df_trial_results_for_regression['WPM']
x = df_trial_results_for_regression[keyboard_categorical_columns]
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WPM   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.5216
Date:                Mon, 11 Mar 2024   Prob (F-statistic):              0.595
Time:                        13:36:26   Log-Likelihood:                -535.47
No. Observations:                 121   AIC:                             1077.
Df Residuals:                     118   BIC:                             1085.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                        144.4644      3.237     44.632      0.000     138.055     150.874
Keyboard_cherry mx2a brown     0.2548      4.550      0.056      0.955      -8.755       9.264
Keyboard_cherry red           -3.9057      4.578     -0.853      0.395     -12.971       5.159
==============================================================================
Omnibus:                        3.907   Durbin-Watson:                   1.645
Prob(Omnibus):                  0.142   Jarque-Bera (JB):                3.803
Skew:                          -0.384   Prob(JB):                        0.149
Kurtosis:                       2.594   Cond. No.                         3.75
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
y = df_trial_results_for_regression['Extra Keystrokes %']
x = df_trial_results_for_regression[keyboard_categorical_columns]
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Extra Keystrokes %   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.6625
Date:                Mon, 11 Mar 2024   Prob (F-statistic):              0.517
Time:                        13:36:26   Log-Likelihood:                -452.33
No. Observations:                 121   AIC:                             910.7
Df Residuals:                     118   BIC:                             919.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          8.0015      1.628      4.914      0.000       4.777      11.226
Keyboard_cherry mx2a brown     2.5261      2.289      1.104      0.272      -2.006       7.058
Keyboard_cherry red            1.9284      2.303      0.837      0.404      -2.631       6.488
==============================================================================
Omnibus:                       58.962   Durbin-Watson:                   1.854
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              185.264
Skew:                           1.834   Prob(JB):                     5.90e-41
Kurtosis:                       7.826   Cond. No.                         3.75
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
y = df_trial_results_for_regression['WPM']
x = df_trial_results_for_regression[
['Extra Keystrokes %']+keyboard_categorical_columns]
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WPM   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.706
Method:                 Least Squares   F-statistic:                     96.89
Date:                Mon, 11 Mar 2024   Prob (F-statistic):           1.42e-31
Time:                        13:36:26   Log-Likelihood:                -460.49
No. Observations:                 121   AIC:                             929.0
Df Residuals:                     117   BIC:                             940.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                        157.8720      1.920     82.235      0.000     154.070     161.674
Extra Keystrokes %            -1.6756      0.099    -16.944      0.000      -1.871      -1.480
Keyboard_cherry mx2a brown     4.4876      2.471      1.816      0.072      -0.406       9.382
Keyboard_cherry red           -0.6745      2.481     -0.272      0.786      -5.588       4.239
==============================================================================
Omnibus:                        0.188   Durbin-Watson:                   1.737
Prob(Omnibus):                  0.910   Jarque-Bera (JB):                0.065
Skew:                          -0.056   Prob(JB):                        0.968
Kurtosis:                       3.020   Cond. No.                         46.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""